# Web app development using Flask

## Flask

* minimal framework, extensible by community extensions
* route
    * syntax through decorator & funcs
* environmental variables
    ```
    export FLASK_APP=app.py
    export FLASK_ENV=development
    ```
    * filename & environment to operate the env
    * can be passed through app arguments

* run
    ```
    flask run
    ```
    * default address `localhost:5000`

* returning json
    * first method
        ```
        from flask import Flask
        app = Flask(meh)

        @app.route('/')
        def index():
            return {"message":"Hello World"}
        ```
    * second method
        ```
        from flask import Flask, jsonify
        app = Flask(meh)

        @app.route('/')
        def index():
            return jsonify(message="Hello World")
        ``` 

* interesting params
    * ENV, DEBUG, TESTING, SECRET_KEY, SESSION_COOKIE_NAME, SERVER_NAME, JSONIFY_MIMETYPE

* configuration object to setup app
    `app.config['SECRET_KEY] = "random-secret-key"`
* configure from an environment variable
    `app.config["VARIABLE_NAME"]`
    `app.config.from_prefixed_env()`
* configure from a python file
    `app.config.from_file("pathtoconfigfile")`

* application structure example
    
    ```
    config.json
    requirements.txt
    setup.py
    src
    |
    |--- __init__.py
    |--- static
        |--- css
            |--- main.css
        |--- img
            |--- header.png
        |--- js
            |--- site.js

    |--- templates
        |--- about.html
        |--- index.html

    |--- tests
        |--- test_auth.py
        |--- test_site.py
    
    |--- venv
    ```

## Request & response objects

* customizing path using path decorator, `@app.route("/health", methods=["GET", "POST"])`
* implementation downstream must reflect the method
* http calls contain the Flask.Request instance
    
    * common request attributes
        * server
        * headers
        * URL
        * access_route
        * full_path
        * is_secure
        * is_JSON
        * cookies

    * additional data
        * Cache-Control
        * Accept
        * Accept-Encoding
        * User-Agent
        * Accept-Language
        * Host

* how to get parsed data (requests)
    * args: MultiDict[str, str]
    * JSON: Optional[Any]
    * files: ImmutableMultiDict[str, FileStorage]
    * form: ImmutableMultiDict[str, str]
    * values: CombinedMultiDict[str, str]
    * dicts can be treated as a data type

* response attributes
    * status_code
    * headers
    * content_type
    * content_length
    * content_encoding
    * mimetype
    * expires

* response common methods
    * set_cookie
    * delete_cookie

* success response through
    * JSONify method
    * make_response method
    * redirect response
    * abort method

### Dynamic routes

* calling external API, easily done through requests lib, return JSON as a dict to the caller

```
from flask import Flask, escape
import requests

app = Flask(__name__)

@app.route("/")
def get_author():
    res = requests.get("https://openlibrary.org/search/authors.JSON?q=Michael Crichton")
    if res.status_code == 200:
        return {"message": res.JSON()}
    elif res.status_code == 404:
        return {"message": "Something went wrong!"}, 404
    else:
        return {"message": "Server error!"}, 500
```

* dynamic routing for passing the params

```
@app.route("/book/<isbn>")
def get_author(isbn):
    res = requests.get("https://openlibrary.org/isbn/{escape(isbn)}.JSON)

    if res.status_code == 200:
        return {"message": res.JSON()}
    elif res.status_code == 404:
        return {"message": "Something went wrong!"}, 404
    else:
        return {"message": "Server error!"}, 500    
```

* examples of dynamic routing through params
    * `@app.route("terminals/<string:airport_code>")`
    * `@app.route("book/<int:isbn>")`
    * other types such as path, uuid


### Error handling

* every response includes status code, that are organized within following categories
    * 1xx -> informational (request received)
    * 2xx -> the request has been received and operation is successful
    * 3xx -> ok and redirected
    * 4xx -> request not proper or cannot be fulfilled
    * 5xx -> server issues

* the app needs to return the appropriate code
```
@app.route("/")
def search_response():
    query = request.args.get("q")

    if not query:
        return {"error_message":"Input parameter missing"}, 422
    resource = fetch_from_database(query)

    if resource:
        return {"message": resource}
    else:
        return {"error_message":"Resource not found"}, 404

```

### Deploying web flask using Flask

* CRUD framework through methods
    * POST -> Create
    * GET -> Read
    * PUT/PATCH -> Update
    * DELETE -> Delete

* creation process
    * install env
    * import package
    * instantiate app
    * run app

* rendering templates
    * to serve static or dynamic HTML content
    * passing arguments

```
from flask import Flask, render_template, request
app = Flask("My first app")

@app.route("/sample")
def getSampleHtml():
    return render_template("sample.html")

@app.route("/user/<username>", methods=["GET"])
def greetUser(username):
    return render_template("result.html", username=username)

@app.route("user", methods=["GET"])
def greetUserBasedOnReq():
    username = request.args.get("username")
    return render_template("result.html", username=username)

if __name__=="__main__":
    app.run(debug=True)
```